<a href="https://colab.research.google.com/github/qamtam/Hands-on-machine-learning/blob/main/CH16_PART2_with_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#stateless vs stateful RNNS
#stateless wyrzuca za przy każdym batchu i przy każdej iteracji i przy każdym epochu stany ukryte w warstwach gru
#w wyniku tego za każdym razem zaczyna od zera

#staateful mieli pierwszy training batch i wykorzystuje to jako stan ukryty na początku przemielenia drugiego batcha itd.
#stateful RNN ma sens tylko i wyłącznie jeśli każdy element batcha zaczyna się tam, gdzie poprzedni się kończy
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

    
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1
train_size = dataset_size * 90 // 100

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead


     |████████████████████████████████| 1.1MB 8.4MB/s 
1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

dataset = dataset.window(window_length, shift = n_steps, drop_remainder = True) # komentarz dziennik II 2020 s 112
dataset = dataset.flat_map(lambda window: window.batch(window_length))

dataset = dataset.repeat().batch(1)

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:])) # pierwsze windows to drugi X_batch i Y_batch w drugiej linijce
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch))

dataset = dataset.prefetch(1)

In [ ]:
for (x, y) in dataset.take(1):
  print(x,y)

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]], shape=(1, 100, 39), dtype=float32) tf.Tensor(
[[ 5  8  7  2  0 18  5  2  5 35  1  9 23 10 21  1 19  3  8  1  0 16  1  0
  22  8  3 18  1  1 12  0  4  9 15  0 19 13  8  2  6  1  8 17  0  6  1  4
   8  0 14  1  0  7 22  1  4 24 26 10 10  4 11 11 23 10  7 22  1  4 24 17
   0  7 22  1  4 24 26 10 10 19  5  8  7  2  0 18  5  2  5 35  1  9 23 10
  15  3 13  0]], shape=(1, 100), dtype=int64)


In [ ]:
#fancy batches
# okno 1 .. 32 (znaki 0 - 32*101) -> batch 1
# okno 2 .. 33 -> batch 2

batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset) # tworzymy listę mikro datasetów
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)


In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])


In [ ]:

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])


Epoch 1/50
313/313 [==============================] - 79s 252ms/step - loss: 2.6145
Epoch 2/50
313/313 [==============================] - 79s 251ms/step - loss: 2.1799
Epoch 3/50
313/313 [==============================] - 78s 248ms/step - loss: 2.5664
Epoch 4/50
313/313 [==============================] - 78s 251ms/step - loss: 2.4927
Epoch 5/50
313/313 [==============================] - 78s 250ms/step - loss: 2.2136
Epoch 6/50
313/313 [==============================] - 78s 250ms/step - loss: 2.1566
Epoch 7/50
313/313 [==============================] - 78s 250ms/step - loss: 2.0477
Epoch 8/50
313/313 [==============================] - 79s 253ms/step - loss: 2.0158
Epoch 9/50
313/313 [==============================] - 80s 254ms/step - loss: 1.9707
Epoch 10/50
313/313 [==============================] - 78s 249ms/step - loss: 1.9354
Epoch 11/50
313/313 [==============================] - 78s 249ms/step - loss: 1.9128
Epoch 12/50
313/313 [==============================] - 78s 248ms/step - lo

In [ ]:
# żeby używać modelu startując z fragmentu dowolnej długości należy najpierw stworzyć stateless copy
# w przeciwnym razie będzie akceptować wyłącznie fragmenty o długości równej dokładnie długości batcha

stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]), #dropout jest zbędny bo i tak jest używany tylko przy treningu
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
stateless_model.build(tf.TensorShape([None, None, max_id])) # model musi zostać zbudowany, żeby ustalić wagi
stateless_model.set_weights(model.get_weights())
model = stateless_model
tf.random.set_seed(42)


def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)
def next_char(text, temperature = 1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :] # pierwszy rząd, ostatni znak, prawdopodobieństwa
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples = 1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature = 1):
  for _ in range(n_chars):
    text += next_char(text, temperature)
  return text
print(complete_text("the dog")) # użycie modelu

your mother:
no doing honour this vens with beaught wonst
th


In [ ]:
print(complete_text("the dog")) # użycie modelu

the dogging broke;
and through conscrieves evension as ty
